# setup

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh','matplotlib')

In [ ]:
%%output backend='matplotlib'
%opts Overlay [legend_position='bottom']
%opts Points.R (color='r') Points.XY (color='blue') Points.XZ (color='green' size=5)

In [ ]:
%%output backend='bokeh'
%opts Overlay [legend_position='bottom']
%opts Points.R (color='r') Points.XY (color='blue') Points.XZ (color='green' size=5)

In [ ]:
def mk_vector(): return np.random.normal(size=100)
df = pd.DataFrame( dict( x = mk_vector(), y=mk_vector(), z=mk_vector()))
for i in range(5): df[str(i)] = mk_vector()

df.tail().round(2)

# example 1

In [ ]:
import logging

def mk_view_1(use_z=True):
    h = hv.Points( df, kdims=['x', 'y'], group='XY', label='xy')*\
        hv.Points( (mk_vector(),mk_vector()), group='R', label='random')
    if use_z:
        h = h*hv.Points(df, kdims=['x', 'z'], group='XZ', label='xz')
        #logging.info('z')
    return h

# in the following, use_z is initialized to 0 - which python interprets as False
dmap_1=hv.DynamicMap(mk_view_1, kdims=['use_z']).redim.values(use_z=[True,False])
dmap_1

In [ ]:
# This initializes use_z correctly, but changing the setting no longer works
hv.HoloMap(dmap_1)

In [ ]:
%%output backend='matplotlib'
# in this example, use_z is initialized to 0, selector can be changed once, has no effect thereafter
plt_dmap_1=hv.DynamicMap(mk_view_1, kdims=['use_z']).redim.values(use_z=[True,False])
plt_dmap_1

# example 2

In [ ]:
%%opts Distribution {+framewise}

def mk_view_2(i, use_z=True):
    h = hv.Points( df, kdims=['x', 'y'], group='XY', label='xy')*\
        hv.Points( (mk_vector(),mk_vector()), group='R', label='random')
    if use_z:
        h = h*hv.Points(df, kdims=['x', 'z'], group='XZ', label='xz')
        #logging.info('z')

    return (h+hv.Distribution( df[str(i)])).relabel('CURRENT i=%d'%i)

#use_z is correctly initialized to true, this example works as expected
#However, the title returned by mk_view_2 stays fixed!
dmap_2=hv.DynamicMap(mk_view_2, kdims=['i', 'use_z']).redim.values(i=np.arange(5), use_z=[True,False])
dmap_2

In [ ]:
%%opts Distribution {+framewise}
# this fails: selectors do not work, trying to change i changes the use_z selector
hv.HoloMap(dmap_2)

In [ ]:
%%opts Distribution {+framewise}
# this fails too:
#  AttributeError: 'NoneType' object has no attribute 'clone'

dmap_3=hv.DynamicMap(mk_view_2, kdims=['i', 'use_z']).redim.values(i=np.arange(5), use_z=[True,False])
hv.HoloMap(dmap_3).collate()

In [ ]:
%%output backend='matplotlib'

# This produces an error:
# Exception: Some Elements returned by the dynamic callback were not initialized correctly and could not be rendered.
plt_dmap_2=hv.DynamicMap(mk_view_2, kdims=['i', 'use_z']).redim.values(i=np.arange(5), use_z=[True,False])
plt_dmap_2

# example 3

In [ ]:
def mk_view_3(i,sigma,unused,use_z=True):
    h=\
    hv.Points( df, kdims=['x', 'y'], group='XY', label='xy')*\
    hv.Points( (df['1'],sigma*df['2']),  group='R', label='random')*\
    hv.Points( df, kdims=[str(i),str(i)],group='XZ', label='diagonal')
    if use_z:
        h = h*hv.Points(df, kdims=['x', 'z'], group='XZ', label='xz')
    return h
dmap_3 = hv.DynamicMap( mk_view_3, kdims=['i','sigma','unused','use_z'])\
          .redim.values(i=np.arange(5),sigma=[2.,3.5,5.], unused=[3,4,5], use_z=[True,False])

# this just produces a text representation:   :HoloMap   [i,sigma,unused,use_z]
hv.HoloMap(dmap_3)

In [ ]:
%%output backend='matplotlib'
plt_dmap_3 = hv.DynamicMap( mk_view_3, kdims=['i','sigma','unused','use_z'])\
          .redim.values(i=np.arange(5),sigma=[2.,3.5,5.], unused=[3,4,5], use_z=[True,False])
# this just produces a text representation:   :HoloMap   [i,sigma,unused,use_z]
hv.HoloMap(plt_dmap_3)

# example 4

In [ ]:
%%opts Histogram Distribution [invert_axes=True] {+framewise}
def mk_view_4(i,unused):
    h=\
    hv.Histogram(np.histogram(df[str(i)],normed=True),label='hist')*\
    hv.Distribution( df[str(i)], group='XY', label='xy')
    return h

dmap_4 = hv.DynamicMap( mk_view_4, kdims=['i','unused'])\
          .redim.values(i=np.arange(5), unused=[3,4,5])

if True:
    dmap = dmap_4
    # no legend for Distribution
    # would be nice to be able to change hist alpha by clicking on hist legend
else:
    dmap = dmap_4 + hv.HoloMap(dmap_4)
    # This does not work
    # interestingly, this displays the plot without inverting axes,
    # then redisplays it a little later in time with inverted axes
dmap

In [ ]:
# this displays, but the selector for i has no effect
hv.HoloMap(dmap_4)

In [ ]:
%%output backend='matplotlib'
plt_dmap_4 = hv.DynamicMap( mk_view_4, kdims=['i','unused'])\
          .redim.values(i=np.arange(5), unused=[3,4,5])

# plt_dmap_4 by itself works as expected
#    but this layout does not
plt_dmap_4 + hv.HoloMap(plt_dmap_4)

I have a weird case with actual data I can't yet reproduce:
> for one of the cases (i = 2), the inverted_axis=2 plot produces a vertical line,
yet when called directly, I get the expected Histogram... I'll try and see about reducing the data to a minimum

# example 5

In [ ]:
%%opts Histogram Distribution [invert_axes=True width=200 yaxis=None] {+framewise}
%%opts Path (color='indianred', line_width=4)
%%opts DynamicMap [width=700]
#      ^^^^^^^^^^             seems to be honored, but produces a warning
def mk_view_5(i,unused):
    points = \
        hv.Points( df, kdims=['x', 'y'], group='XY', label='xy') *\
        hv.Points( (mk_vector(),mk_vector()), group='R', label='random')
    lines  = hv.Path([ [(-1, -1), (2,2), (-1,3), (3,-1), (3,2)] ],label='path')
    #                                                             ^^^^^^^^^^^^  legend not honored
    
    hist = hv.Histogram(np.histogram(df[str(i)],normed=True))*\
           hv.Distribution( df[str(i)], group='XY')

    return points*lines+hist
    #return points << hist  # does not work by itself

dmap_5 = hv.DynamicMap( mk_view_5, kdims=['i','unused'])\
          .redim.values(i=np.arange(5),unused=[0])
dmap_5

In [ ]:
%%output backend='matplotlib' size=100
%%opts Histogram Distribution [invert_axes=True width=200 yaxis=None] {+framewise}
%%opts Path (color='indianred', line_width=4)

# with matplotlib, the smae graph works without any warnings!

plt_dmap_5 = hv.DynamicMap( mk_view_5, kdims=['i','unused'])\
          .redim.values(i=np.arange(5),unused=[0])
plt_dmap_5

# example 6

In [ ]:
%%opts Contours [width=800 colorbar=True colorbar_position='left']
%%opts Image (cmap='Blues_r')
vals=np.linspace(-10,10, 200); xx,yy=np.meshgrid(vals,vals)
img=hv.Image(xx*yy)
img*hv.operation.contours(img)
# why did this generate legends?  hv.operation.contours(img) does not
# why are the legends not clickable individually, why does clicking them have no effect?

In [ ]:
%%output backend='matplotlib' size=200
%%opts Contours [width=800 colorbar=True colorbar_position='left']
%%opts Image (cmap='Blues_r')
vals=np.linspace(-10,10, 200); xx,yy=np.meshgrid(vals,vals)
img=hv.Image(xx*yy)
img*hv.operation.contours(img)

# no legends here, but the colorbar_position is not honored

# example 7

In [ ]:
# %%opts Points [width=500 tools=['hover']] (size=5 color=Cycle('Blues'))

# cannot get points to honor the color!

import seaborn as sns
iris            = sns.load_dataset("iris")
iris['species'] = iris['species'].astype('category')

def strip_plot(df,x,y,jitter=.2):
    ticks = [(i,v) for i,v in enumerate(df[x].cat.categories)]
    return hv.Points(np.array([df[x].cat.codes+np.random.uniform(-jitter,jitter,size=df.shape[0]), df[y], df[x].cat.codes.astype(float)]).T, kdims=[x,y])\
             .opts(plot=dict(xticks=ticks, color_index=x), style=dict(size=5,color=hv.Cycle('Blues')))

strip_plot( iris, 'species', 'sepal_length').redim.range(species=(-.5,2.5),sepal_length=(4,8.5))

In [ ]:
%%output backend='matplotlib' size=200

def strip_plot(df,x,y,jitter=.2):
    ticks = [(i,v) for i,v in enumerate(df[x].cat.categories)]
    return hv.Points(np.array([df[x].cat.codes+np.random.uniform(-jitter,jitter,size=df.shape[0]), df[y], df[x].cat.codes.astype(float)]).T, kdims=[x,y])\
             .opts(plot=dict(xticks=ticks, color_index=x), style=dict(size=5,color=hv.Cycle('Blues')))

strip_plot( iris, 'species', 'sepal_length').redim.range(species=(-.5,2.5),sepal_length=(4,8.5))

# this seems to work apart from the color cycle selection

In [ ]:
# BoxWhisker does not accept ticks lists: the x axis ticks are the cat.codes rather than the cat.names

def boxwhisker_plot(df,x,y,jitter=.2):
    ticks = [(i,v) for i,v in enumerate(df[x].cat.categories)]
    return hv.BoxWhisker(np.array([df[x].cat.codes, df[y]]).T, x,y)\
             .opts(plot=dict(xticks=ticks))
boxwhisker_plot( iris, 'species', 'sepal_length').redim.range(species=(-.5,2.5),sepal_length=(4,8.5))

In [ ]:
%%output backend='matplotlib'

def boxwhisker_plot(df,x,y,jitter=.2):
    ticks = [(i,v) for i,v in enumerate(df[x].cat.categories)]
    return hv.BoxWhisker(np.array([df[x].cat.codes, df[y]]).T, x,y)\
             .opts(plot=dict(xticks=ticks, color_index=x))
boxwhisker_plot( iris, 'species', 'sepal_length').redim.range(species=(-.5,2.5),sepal_length=(4,8.5))

# here, the x axis is messed up

# example 8

In [ ]:
%%opts Bivariate [bandwidth=0.5] (cmap='Blues') Points (s=4)
from bokeh.sampledata.iris import flowers
from holoviews.operation import gridmatrix

if True:
    # gridmatrix cannot handle categorical entries
    flowers['species'] = flowers['species'].astype('category')
elif False:
    # yields: 'Image dimension petal_length is not evenly sampled, please use the QuadMesh' error message
    flowers['species'] =100.* (flowers['species'].astype('category').cat.codes+1).astype(float)


iris_ds = hv.Dataset(flowers)

density_grid = gridmatrix(iris_ds, diagonal_type=hv.Distribution, chart_type=hv.Bivariate)
point_grid   = gridmatrix(iris_ds, diagonal_type=hv.Distribution, chart_type=hv.Points   )
point_grid = point_grid.map(lambda x: hv.Overlay(), hv.Distribution)

density_grid * point_grid

# example 9

In [42]:
%%opts Overlay [width=100 height=100]
%%opts Points (color=Cycle(['#ff00ff','#00ff00','#0000ff']))

iris = flowers
def gen_grid_func(df,v):
    color_index = 'species'
    if False:
        df = df.copy()
        df['color'] = (df[v].astype('category').cat.codes+1).astype(float)
        color_index
        # using color_index='color' below results in 'singular matrix' error

    N = df.shape[1]
    def top(i,j):
        h = \
        hv.Bivariate(df, kdims=[df.columns[i],df.columns[j]])*\
        hv.Points   (df, kdims=[df.columns[i],df.columns[j]]).opts(plot=dict(color_index='species'))
        return h
    def diag(i):
        s = df[v].unique()
        h = hv.Overlay([ hv.Distribution( df[df['species']==j], df.columns[i]) for j in s])
        return h
    def btm(i,j):
        h = \
        hv.Overlay([hv.Points   (df, kdims=[df.columns[i],df.columns[j]])])
        return h

    plots  = {(i,j): top(i,j) for i in range(N) for j in range(i+1,N) if df.columns[i] != v and df.columns[j] != v}
    dplots = {(i,i): diag(i ) for i in range(N) if df.columns[i] != v }
    bplots = {(i,j): btm(i,j) for i in range(N) for j in range(i) if df.columns[i] != v and df.columns[j] != v}

    plots.update(dplots)
    plots.update(bplots)
    
    return plots

# GridMatrix does not honor xticks, yticks
# can't figure out how to control point colors: should be the same as the distributions.
# Want a legend for the species columns (ideally mutable!)
ticks = [i for i in iris.columns if i != 'species']
hv.GridSpace(gen_grid_func(iris, 'species'), kdims=['measurement','measurement']).opts( plot=dict(xticks=ticks, yticks=ticks)).relabel('Measurements by species')

:GridSpace   [measurement,measurement]
   :Overlay
      .Distribution.I   :Distribution   [petal_width]   (Density)
      .Distribution.II  :Distribution   [petal_width]   (Density)
      .Distribution.III :Distribution   [petal_width]   (Density)